In [9]:
# Imports + mixed precision + device info
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Mixed precision
mixed_precision.set_global_policy('mixed_float16')

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input

print("TF:", tf.__version__)
print("Physical devices:", tf.config.list_physical_devices())
print("GPU devices (MPS):", tf.config.list_physical_devices('GPU'))

TF: 2.16.1
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU devices (MPS): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
# Paths and config
DATA_TRAIN_DIR = "../data/Training"   
DATA_TEST_DIR  = "../data/Testing"    

IMG_SIZE = (224, 224)
BATCH_SIZE = 16     
SEED = 42
NUM_CLASSES = None  # will infer from generator
EPOCHS = 40

In [11]:
# Generators (uses preprocess_input for ResNet)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.15,   # holdout val from training folder
    rotation_range=10,
    width_shift_range=0.03,
    height_shift_range=0.03,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    DATA_TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)
val_gen = train_datagen.flow_from_directory(
    DATA_TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED
)
test_gen = test_datagen.flow_from_directory(
    DATA_TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

NUM_CLASSES = train_gen.num_classes
print("Classes:", train_gen.class_indices)
print("Train/Val/Test sample counts:", train_gen.samples, val_gen.samples, test_gen.samples)

Found 4857 images belonging to 4 classes.
Found 855 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Classes: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
Train/Val/Test sample counts: 4857 855 1311


In [16]:
# Build ResNet50 based classification model 
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

def build_resnet_classifier(input_shape=(224, 224, 3), num_classes=4, encoder_weights='imagenet', trainable_encoder=False):
    base_model = ResNet50(include_top=False, weights=encoder_weights, input_shape=input_shape)
    base_model.trainable = trainable_encoder  # can unfreeze for fine tuning

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)  # ensures float32 output

    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

model = build_resnet_classifier(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    num_classes=NUM_CLASSES,
    encoder_weights='imagenet',
    trainable_encoder=False
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,122,500 (92.02 MB)

 Trainable params: 530,180 (2.02 MB)

 Non-trainable params: 23,592,320 (90.00 MB)

In [17]:
# Compile + callbacks
opt = optimizers.Adam(learning_rate=2e-4)  # ReduceLR will adjust
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1),
    ModelCheckpoint('resnet_unet_best.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
]

In [19]:
# Training

history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=EPOCHS,
    callbacks=callbacks,
)

Epoch 1/40
303/303 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.8990 - loss: 0.2781
Epoch 1: val_accuracy improved from 0.88090 to 0.91156, saving model to resnet_unet_best.keras
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 142ms/step - accuracy: 0.8905 - loss: 0.2972 - val_accuracy: 0.9116 - val_loss: 0.2639 - learning_rate: 2.0000e-04
Epoch 2/40
  1/303 ━━━━━━━━━━━━━━━━━━━━ 35s 119ms/step - accuracy: 1.0000 - loss: 0.0657
Epoch 2: val_accuracy did not improve from 0.91156
303/303 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 1.0000 - loss: 0.0657 - val_accuracy: 0.9116 - val_loss: 0.2639 - learning_rate: 2.0000e-04
Epoch 3/40
303/303 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.8956 - loss: 0.2937
Epoch 3: val_accuracy did not improve from 0.91156
303/303 ━━━━━━━━━━━━━━━━━━━━ 43s 141ms/step - accuracy: 0.8973 - loss: 0.2890 - val_accuracy: 0.9033 - val_loss: 0.2888 - learning_rate: 2.0000e-04
Epoch 4/40
  1/303 ━━━━━━━━━━━━━━━━━━━━ 37s 124ms/step - accuracy: 0.8125 - loss: 0.4702
Epoch 4

In [21]:
# Unfreezing last residual block of ResNet50
for layer in model.layers:
    if "conv5_block" in layer.name:
        layer.trainable = True

# Compiling with a smaller LR for fine-tuning
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Fine-tuning: last ResNet block unfrozen ")

Fine-tuning: last ResNet block unfrozen 


In [22]:
fine_tune_epochs = 10  
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=fine_tune_epochs,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.8667 - loss: 0.3712
Epoch 1: val_accuracy did not improve from 0.91156
304/304 ━━━━━━━━━━━━━━━━━━━━ 71s 215ms/step - accuracy: 0.8824 - loss: 0.3268 - val_accuracy: 0.9029 - val_loss: 0.2785 - learning_rate: 1.0000e-05
Epoch 2/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.9045 - loss: 0.2569
Epoch 2: val_accuracy improved from 0.91156 to 0.91345, saving model to resnet_unet_best.keras
304/304 ━━━━━━━━━━━━━━━━━━━━ 65s 215ms/step - accuracy: 0.9111 - loss: 0.2420 - val_accuracy: 0.9135 - val_loss: 0.2598 - learning_rate: 1.0000e-05
Epoch 3/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 0.9256 - loss: 0.2033
Epoch 3: val_accuracy improved from 0.91345 to 0.92164, saving model to resnet_unet_best.keras
304/304 ━━━━━━━━━━━━━━━━━━━━ 80s 264ms/step - accuracy: 0.9281 - loss: 0.1986 - val_accuracy: 0.9216 - val_loss: 0.2166 - learning_rate: 1.0000e-05
Epoch 4/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/st

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluation on test set
test_loss, test_acc = model.evaluate(val_gen)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# Predictions
y_true = val_gen.classes
y_pred = np.argmax(model.predict(val_gen), axis=1)

print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred, digits=4))

54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 106ms/step - accuracy: 0.9556 - loss: 0.1337
Test Accuracy: 0.9556, Test Loss: 0.1337
54/54 ━━━━━━━━━━━━━━━━━━━━ 9s 136ms/step

Confusion Matrix:
 [[195   3   0   0]
 [  3 190   1   6]
 [  1   5 227   6]
 [  0   3   2 213]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9799    0.9848    0.9824       198
           1     0.9453    0.9500    0.9476       200
           2     0.9870    0.9498    0.9680       239
           3     0.9467    0.9771    0.9616       218

    accuracy                         0.9649       855
   macro avg     0.9647    0.9654    0.9649       855
weighted avg     0.9653    0.9649    0.9649       855

